# **`10. Analisis Matriks Korelasi K-mer: Sebelum dan Sesudah Dekomposisi PCA`**

Bagian ini membandingkan struktur korelasi fitur k-mer sebelum dan sesudah reduksi dimensi menggunakan PCA, untuk memvalidasi bahwa PCA berhasil menghilangkan multikolinearitas.

## **10.1 Import Required Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## **10.2 Load Original K-mer Data (Before PCA)**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import load_npz
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# ============================================
# STEP 1: LOAD DATA
# ============================================

level_used = "species"
k_used = "k6"
method_used = "ipca"

print("="*80)
print("📂 LOADING DATA")
print("="*80)

# ✅ FIX 1: Load NPZ (sparse matrix format)
original_train_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/prep/vectorization/{level_used}/{k_used}/X_train_sparse_{k_used}_{level_used}.npz"

print(f"\n🔄 Loading original k-mer data (sparse NPZ)...")
print(f"   Path: {original_train_path}\n")

try:
    # Load sparse matrix
    X_train_sparse = load_npz(original_train_path)
    print(f"✅ Sparse matrix loaded:")
    print(f"   Type: {type(X_train_sparse)}")
    print(f"   Shape: {X_train_sparse.shape}")
    print(f"   Sparsity: {1 - X_train_sparse.nnz / (X_train_sparse.shape[0] * X_train_sparse.shape[1]):.2%}")
    
    # Convert to dense for correlation (hanya sample fitur dengan variance tinggi)
    print(f"\n🔄 Converting to dense & sampling high-variance features...")
    X_dense = X_train_sparse.toarray()
    
    # Sample top features by variance
    feature_variance = np.var(X_dense, axis=0)
    n_features_sample = min(100, X_dense.shape[1])  # Sample 100 features max
    top_feature_indices = np.argsort(-feature_variance)[:n_features_sample]
    
    df_original = pd.DataFrame(X_dense[:, top_feature_indices])
    df_original.columns = [f'KMer_{i}' for i in top_feature_indices]
    
    print(f"✅ Original data sampled:")
    print(f"   Shape: {df_original.shape}")
    print(f"   Features selected: {n_features_sample} (highest variance)")
    
except Exception as e:
    print(f"❌ Error loading sparse matrix: {e}")
    raise

# ✅ FIX 2: Load PCA-reduced data
pca_train_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/features_fold0_train.csv"

print(f"\n🔄 Loading PCA-reduced data...")
print(f"   Path: {pca_train_path}\n")

try:
    df_pca = pd.read_csv(pca_train_path)
    
    # Remove label column if exists
    if 'label' in df_pca.columns:
        df_pca = df_pca.drop(columns=['label'])
    
    print(f"✅ PCA data loaded:")
    print(f"   Shape: {df_pca.shape}")
    print(f"   Dimensionality reduction: {df_original.shape[1]} → {df_pca.shape[1]} ({(1 - df_pca.shape[1]/df_original.shape[1])*100:.1f}%)")
    
except Exception as e:
    print(f"❌ Error loading PCA data: {e}")
    raise

# ============================================
# STEP 2: COMPUTE CORRELATION MATRICES
# ============================================

print("\n" + "="*80)
print("🔄 COMPUTING CORRELATION MATRICES")
print("="*80)

print("\n📊 Original K-mer features...")
corr_original = df_original.corr(method='pearson')
upper_tri_original = corr_original.values[np.triu_indices_from(corr_original.values, k=1)]

print(f"✅ Correlation matrix (Original):")
print(f"   Shape: {corr_original.shape}")
print(f"   Mean |r|: {np.abs(upper_tri_original).mean():.4f}")
print(f"   High corr (|r| > 0.7): {(np.abs(upper_tri_original) > 0.7).sum() / len(upper_tri_original) * 100:.2f}%")

print("\n📊 PCA components...")
corr_pca = df_pca.corr(method='pearson')
upper_tri_pca = corr_pca.values[np.triu_indices_from(corr_pca.values, k=1)]

print(f"✅ Correlation matrix (PCA):")
print(f"   Shape: {corr_pca.shape}")
print(f"   Mean |r|: {np.abs(upper_tri_pca).mean():.4f}")
print(f"   High corr (|r| > 0.7): {(np.abs(upper_tri_pca) > 0.7).sum() / len(upper_tri_pca) * 100:.2f}%")

# ============================================
# STEP 3: NATURE-STYLE HEATMAP COMPARISON
# ============================================

print("\n" + "="*80)
print("🎨 CREATING NATURE-STYLE PLOTS")
print("="*80)

# Set Nature publication style
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.linewidth'] = 1.2

# ─────────────────────────────────────────
# PANEL A & B: Side-by-side heatmaps
# ─────────────────────────────────────────

fig = plt.figure(figsize=(18, 8))

# ✅ PANEL A: Original K-mer Correlations
ax1 = plt.subplot(1, 2, 1)

im1 = ax1.imshow(corr_original.values, cmap='RdBu_r', aspect='auto', 
                  vmin=-1, vmax=1, interpolation='nearest')

ax1.set_title('A. K-mer Features (Original)', 
             fontsize=13, fontweight='bold', loc='left', pad=10)
ax1.set_xlabel('K-mer Feature Index', fontsize=11, fontweight='bold')
ax1.set_ylabel('K-mer Feature Index', fontsize=11, fontweight='bold')

# Reduce tick labels untuk readability
n_ticks_orig = min(10, corr_original.shape[0])
tick_positions_orig = np.linspace(0, corr_original.shape[0]-1, n_ticks_orig, dtype=int)
ax1.set_xticks(tick_positions_orig)
ax1.set_yticks(tick_positions_orig)
ax1.set_xticklabels([str(i) for i in tick_positions_orig], fontsize=9)
ax1.set_yticklabels([str(i) for i in tick_positions_orig], fontsize=9)

# Add colorbar
cbar1 = plt.colorbar(im1, ax=ax1, label='Pearson r')
cbar1.ax.tick_params(labelsize=9)

# Add text annotation with stats
stats_text_orig = (
    f"Features: {corr_original.shape[0]}\n"
    f"Mean |r|: {np.abs(upper_tri_original).mean():.3f}\n"
    f"High corr: {(np.abs(upper_tri_original) > 0.7).sum() / len(upper_tri_original) * 100:.1f}%"
)
ax1.text(0.98, 0.02, stats_text_orig, 
        transform=ax1.transAxes,
        fontsize=9, verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='gray', linewidth=1),
        fontfamily='monospace')

# ✅ PANEL B: PCA Components Correlations
ax2 = plt.subplot(1, 2, 2)

im2 = ax2.imshow(corr_pca.values, cmap='RdBu_r', aspect='auto',
                  vmin=-1, vmax=1, interpolation='nearest')

ax2.set_title('B. Principal Components (After IPCA)', 
             fontsize=13, fontweight='bold', loc='left', pad=10)
ax2.set_xlabel('PC Index', fontsize=11, fontweight='bold')
ax2.set_ylabel('PC Index', fontsize=11, fontweight='bold')

# Tick labels untuk PCA
n_ticks_pca = min(10, corr_pca.shape[0])
tick_positions_pca = np.linspace(0, corr_pca.shape[0]-1, n_ticks_pca, dtype=int)
ax2.set_xticks(tick_positions_pca)
ax2.set_yticks(tick_positions_pca)
ax2.set_xticklabels([str(i) for i in tick_positions_pca], fontsize=9)
ax2.set_yticklabels([str(i) for i in tick_positions_pca], fontsize=9)

# Add colorbar
cbar2 = plt.colorbar(im2, ax=ax2, label='Pearson r')
cbar2.ax.tick_params(labelsize=9)

# Add text annotation with stats
stats_text_pca = (
    f"Components: {corr_pca.shape[0]}\n"
    f"Mean |r|: {np.abs(upper_tri_pca).mean():.3f}\n"
    f"High corr: {(np.abs(upper_tri_pca) > 0.7).sum() / len(upper_tri_pca) * 100:.1f}%"
)
ax2.text(0.98, 0.02, stats_text_pca,
        transform=ax2.transAxes,
        fontsize=9, verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='gray', linewidth=1),
        fontfamily='monospace')

# Overall title
fig.suptitle(f'Correlation Structure: Before vs After Dimensionality Reduction\n'
            f'{level_used.upper()} K{k_used} ({method_used.upper()})',
            fontsize=15, fontweight='bold', y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.96])

# Save
output_path_heatmap = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_matrix_comparison_heatmap.png"
plt.savefig(output_path_heatmap, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✅ Heatmap saved: {output_path_heatmap}")
plt.close()

# ============================================
# STEP 4: DISTRIBUTION COMPARISON (Nature Style)
# ============================================

fig = plt.figure(figsize=(14, 10))

# ✅ PANEL A: Histogram comparison
ax1 = plt.subplot(2, 2, 1)

bins = np.linspace(-1, 1, 41)
ax1.hist(upper_tri_original, bins=bins, alpha=0.6, label='Original K-mer', 
        color='#2E86AB', edgecolor='black', linewidth=1)
ax1.hist(upper_tri_pca, bins=bins, alpha=0.6, label='PCA Components',
        color='#A23B72', edgecolor='black', linewidth=1)

ax1.axvline(upper_tri_original.mean(), color='#2E86AB', linestyle='--', linewidth=2.5)
ax1.axvline(upper_tri_pca.mean(), color='#A23B72', linestyle='--', linewidth=2.5)
ax1.axvline(0, color='gray', linestyle='-', linewidth=1, alpha=0.3)

ax1.set_title('A. Correlation Distribution', fontsize=12, fontweight='bold', loc='left')
ax1.set_xlabel('Pearson Correlation Coefficient (r)', fontsize=10, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=10, fontweight='bold')
ax1.legend(loc='upper right', frameon=True, framealpha=0.95, edgecolor='gray')
ax1.grid(True, alpha=0.2)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

# ✅ PANEL B: Absolute correlation comparison
ax2 = plt.subplot(2, 2, 2)

ax2.hist(np.abs(upper_tri_original), bins=bins, alpha=0.6, label='Original K-mer',
        color='#2E86AB', edgecolor='black', linewidth=1)
ax2.hist(np.abs(upper_tri_pca), bins=bins, alpha=0.6, label='PCA Components',
        color='#A23B72', edgecolor='black', linewidth=1)

ax2.axvline(np.abs(upper_tri_original).mean(), color='#2E86AB', linestyle='--', linewidth=2.5,
           label=f"Mean (Original): {np.abs(upper_tri_original).mean():.3f}")
ax2.axvline(np.abs(upper_tri_pca).mean(), color='#A23B72', linestyle='--', linewidth=2.5,
           label=f"Mean (PCA): {np.abs(upper_tri_pca).mean():.3f}")

ax2.set_title('B. Absolute Correlation Distribution', fontsize=12, fontweight='bold', loc='left')
ax2.set_xlabel('|Pearson r|', fontsize=10, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=10, fontweight='bold')
ax2.legend(loc='upper right', fontsize=9, frameon=True, framealpha=0.95, edgecolor='gray')
ax2.grid(True, alpha=0.2)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

# ✅ PANEL C: Boxplot comparison
ax3 = plt.subplot(2, 2, 3)

data_to_plot = [upper_tri_original, upper_tri_pca]
bp = ax3.boxplot(data_to_plot, labels=['Original K-mer', 'PCA Components'],
                 patch_artist=True, notch=True,
                 boxprops=dict(linewidth=1.5),
                 medianprops=dict(color='red', linewidth=2.5),
                 whiskerprops=dict(linewidth=1.5),
                 capprops=dict(linewidth=1.5))

# Color boxes
for patch, color in zip(bp['boxes'], ['#2E86AB', '#A23B72']):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax3.set_title('C. Correlation Coefficient Distribution', fontsize=12, fontweight='bold', loc='left')
ax3.set_ylabel('Pearson r', fontsize=10, fontweight='bold')
ax3.grid(True, alpha=0.2, axis='y')
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)

# ✅ PANEL D: Summary statistics table
ax4 = plt.subplot(2, 2, 4)
ax4.axis('off')

# Create statistics table
stats_data = [
    ['Metric', 'Original K-mer', 'PCA Components', 'Δ (Reduction)'],
    ['─'*15, '─'*18, '─'*18, '─'*15],
    ['Mean r', f"{upper_tri_original.mean():>6.4f}", f"{upper_tri_pca.mean():>6.4f}", 
     f"{(upper_tri_original.mean() - upper_tri_pca.mean()):>6.4f}"],
    ['Mean |r|', f"{np.abs(upper_tri_original).mean():>6.4f}", 
     f"{np.abs(upper_tri_pca).mean():>6.4f}",
     f"{(np.abs(upper_tri_original).mean() - np.abs(upper_tri_pca).mean()):>6.4f}"],
    ['Median |r|', f"{np.median(np.abs(upper_tri_original)):>6.4f}",
     f"{np.median(np.abs(upper_tri_pca)):>6.4f}",
     f"{(np.median(np.abs(upper_tri_original)) - np.median(np.abs(upper_tri_pca))):>6.4f}"],
    ['Std |r|', f"{np.std(np.abs(upper_tri_original)):>6.4f}",
     f"{np.std(np.abs(upper_tri_pca)):>6.4f}",
     f"{(np.std(np.abs(upper_tri_original)) - np.std(np.abs(upper_tri_pca))):>6.4f}"],
    ['─'*15, '─'*18, '─'*18, '─'*15],
    ['High corr (>0.7)', 
     f"{(np.abs(upper_tri_original) > 0.7).sum() / len(upper_tri_original) * 100:>5.1f}%",
     f"{(np.abs(upper_tri_pca) > 0.7).sum() / len(upper_tri_pca) * 100:>5.1f}%",
     f"{((np.abs(upper_tri_original) > 0.7).sum() / len(upper_tri_original) - (np.abs(upper_tri_pca) > 0.7).sum() / len(upper_tri_pca)) * 100:>5.1f}%"],
]

# Format as text
table_text = '\n'.join([' | '.join(row) for row in stats_data])

ax4.text(0.05, 0.95, 
        f"📊 STATISTICAL SUMMARY\n\n{table_text}",
        transform=ax4.transAxes,
        fontsize=9, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='#F8F9FA', edgecolor='#CED4DA', 
                 linewidth=2, pad=15))

# Overall title
fig.suptitle('Correlation Structure Analysis: Impact of Dimensionality Reduction',
            fontsize=14, fontweight='bold', y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.97])

# Save
output_path_dist = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_distribution_analysis.png"
plt.savefig(output_path_dist, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✅ Distribution analysis saved: {output_path_dist}")
plt.close()

print("\n" + "="*80)
print("✅ ALL PLOTS GENERATED SUCCESSFULLY")
print("="*80)
print(f"\n📁 Output files:")
print(f"   1. {output_path_heatmap}")
print(f"   2. {output_path_dist}")

📂 LOADING DATA

🔄 Loading original k-mer data (sparse NPZ)...
   Path: /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/prep/vectorization/species/k6/X_train_sparse_k6_species.npz

✅ Sparse matrix loaded:
   Type: <class 'scipy.sparse._csr.csr_matrix'>
   Shape: (54585, 12760)
   Sparsity: 96.98%

🔄 Converting to dense & sampling high-variance features...
✅ Original data sampled:
   Shape: (54585, 100)
   Features selected: 100 (highest variance)

🔄 Loading PCA-reduced data...
   Path: /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/species/k6/ipca/features_fold0_train.csv

✅ PCA data loaded:
   Shape: (54585, 1047)
   Dimensionality reduction: 100 → 1047 (-947.0%)

🔄 COMPUTING CORRELATION MATRICES

📊 Original K-mer features...
✅ Correlation matrix (Original):
   Shape: (100, 100)
   Mean |r|: 0.2122
   High corr (|r| > 0.7): 0.79%

📊 PCA components...
✅ Correlation matrix (PCA):
   Shape: (1047, 1047)
   Mean |r|: 0.0000
   High corr (|r| > 0.7): 0.00%

🎨 CREATING NATURE-STYLE PL

findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

✅ Heatmap saved: /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/species/k6/ipca/correlation_matrix_comparison_heatmap.png


findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial, Helvetica
findfont: Generic family 'sans-serif' not found because none of the foll

✅ Distribution analysis saved: /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/species/k6/ipca/correlation_distribution_analysis.png

✅ ALL PLOTS GENERATED SUCCESSFULLY

📁 Output files:
   1. /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/species/k6/ipca/correlation_matrix_comparison_heatmap.png
   2. /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/species/k6/ipca/correlation_distribution_analysis.png


In [3]:
# Konfigurasi
level_used = "species"
k_used = "k4"  # Ganti sesuai kebutuhan: k4, k6, atau k8
method_used = "ipca"

# Path data asli (sebelum reduksi)
original_train_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/prep/vectorization/species/k4/X_train_sparse_k4_species.npz"

print(f"📂 Loading original k-mer data from:\n   {original_train_path}\n")

# Load data asli
df_original = pd.read_csv(original_train_path)

print(f"✅ Original data loaded:")
print(f"   Shape: {df_original.shape}")
print(f"   Features: {df_original.shape[1]} k-mer features")
print(f"   Samples: {df_original.shape[0]} sequences")
print(f"\n📊 Sample data:")
print(df_original.head())

📂 Loading original k-mer data from:
   /home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/prep/vectorization/species/k4/X_train_sparse_k4_species.npz



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc5 in position 16: invalid continuation byte

## **10.3 Load PCA-Reduced Data (After PCA)**

In [ ]:
# Path data setelah PCA
pca_train_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/features_fold0_train.csv"

print(f"📂 Loading PCA-reduced data from:\n   {pca_train_path}\n")

# Load data PCA
df_pca = pd.read_csv(pca_train_path)

# Remove label column if exists
if 'label' in df_pca.columns:
    df_pca = df_pca.drop(columns=['label'])

print(f"✅ PCA-reduced data loaded:")
print(f"   Shape: {df_pca.shape}")
print(f"   Components: {df_pca.shape[1]} principal components")
print(f"   Samples: {df_pca.shape[0]} sequences")
print(f"   Dimensionality reduction: {df_original.shape[1]} → {df_pca.shape[1]} ({(1 - df_pca.shape[1]/df_original.shape[1])*100:.1f}% reduction)")
print(f"\n📊 Sample data:")
print(df_pca.head())

## **10.4 Calculate Correlation Matrix - Original Data**

In [ ]:
print("🔄 Computing correlation matrix for original k-mer features...")
print(f"   This may take time for {df_original.shape[1]} features...\n")

# Untuk data besar, ambil sample fitur untuk visualisasi
if df_original.shape[1] > 100:
    print(f"⚠️  Too many features ({df_original.shape[1]}) for clear visualization")
    print(f"   Sampling 100 random features for correlation analysis\n")
    
    # Sample fitur dengan variasi tertinggi
    feature_variance = df_original.var().sort_values(ascending=False)
    top_features = feature_variance.head(100).index
    df_original_sample = df_original[top_features]
else:
    df_original_sample = df_original

# Hitung matriks korelasi
corr_matrix_original = df_original_sample.corr(method='pearson')

print(f"✅ Correlation matrix computed:")
print(f"   Matrix shape: {corr_matrix_original.shape}")
print(f"   Mean absolute correlation: {np.abs(corr_matrix_original.values[np.triu_indices_from(corr_matrix_original.values, k=1)]).mean():.4f}")
print(f"   Max correlation (off-diagonal): {np.abs(corr_matrix_original.values[np.triu_indices_from(corr_matrix_original.values, k=1)]).max():.4f}")

# Hitung persentase korelasi tinggi
high_corr_threshold = 0.7
upper_triangle = corr_matrix_original.values[np.triu_indices_from(corr_matrix_original.values, k=1)]
high_corr_pct = (np.abs(upper_triangle) > high_corr_threshold).sum() / len(upper_triangle) * 100
print(f"   High correlation pairs (|r| > {high_corr_threshold}): {high_corr_pct:.2f}%")

## **10.5 Calculate Correlation Matrix - PCA-Reduced Data**

In [ ]:
print("🔄 Computing correlation matrix for PCA components...\n")

# Hitung matriks korelasi PCA
corr_matrix_pca = df_pca.corr(method='pearson')

print(f"✅ Correlation matrix computed:")
print(f"   Matrix shape: {corr_matrix_pca.shape}")
print(f"   Mean absolute correlation: {np.abs(corr_matrix_pca.values[np.triu_indices_from(corr_matrix_pca.values, k=1)]).mean():.4f}")
print(f"   Max correlation (off-diagonal): {np.abs(corr_matrix_pca.values[np.triu_indices_from(corr_matrix_pca.values, k=1)]).max():.4f}")

# Hitung persentase korelasi tinggi
upper_triangle_pca = corr_matrix_pca.values[np.triu_indices_from(corr_matrix_pca.values, k=1)]
high_corr_pct_pca = (np.abs(upper_triangle_pca) > high_corr_threshold).sum() / len(upper_triangle_pca) * 100
print(f"   High correlation pairs (|r| > {high_corr_threshold}): {high_corr_pct_pca:.2f}%")
print(f"\n💡 PCA successfully reduced multicollinearity!")

## **10.6 Visualize Correlation Heatmap - Original K-mer Data**

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(12, 10))

# Plot heatmap
sns.heatmap(
    corr_matrix_original,
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0,
    cbar_kws={
        "shrink": 0.8,
        "label": "Pearson Correlation Coefficient"
    },
    ax=ax
)

# Styling
ax.set_title(
    f'Correlation Matrix - Original K-mer Features ({k_used})\n'
    f'Level: {level_used.capitalize()} | Features: {corr_matrix_original.shape[0]} | '
    f'Mean |r|: {np.abs(upper_triangle).mean():.3f}',
    fontsize=14,
    fontweight='bold',
    pad=20
)

ax.set_xlabel('K-mer Features', fontsize=11, fontweight='bold')
ax.set_ylabel('K-mer Features', fontsize=11, fontweight='bold')

# Rotate labels untuk readability
plt.xticks(rotation=90, ha='right', fontsize=6)
plt.yticks(rotation=0, fontsize=6)

plt.tight_layout()

# Save figure
output_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_matrix_original.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"💾 Figure saved: {output_path}")

plt.show()

## **10.7 Visualize Correlation Heatmap - PCA-Reduced Data**

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(12, 10))

# Plot heatmap
sns.heatmap(
    corr_matrix_pca,
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={
        "shrink": 0.8,
        "label": "Pearson Correlation Coefficient"
    },
    annot=corr_matrix_pca.shape[0] < 20,  # Annotate jika komponen < 20
    fmt='.2f',
    annot_kws={'size': 8},
    ax=ax
)

# Styling
ax.set_title(
    f'Correlation Matrix - PCA Components ({k_used})\n'
    f'Level: {level_used.capitalize()} | Components: {corr_matrix_pca.shape[0]} | '
    f'Mean |r|: {np.abs(upper_triangle_pca).mean():.3f}',
    fontsize=14,
    fontweight='bold',
    pad=20
)

ax.set_xlabel('Principal Components', fontsize=11, fontweight='bold')
ax.set_ylabel('Principal Components', fontsize=11, fontweight='bold')

# Rotate labels
plt.xticks(rotation=90, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()

# Save figure
output_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_matrix_pca.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"💾 Figure saved: {output_path}")

plt.show()

## **10.8 Compare Correlation Distributions**

In [ ]:
# Extract upper triangle values (exclude diagonal)
upper_tri_original = corr_matrix_original.values[np.triu_indices_from(corr_matrix_original.values, k=1)]
upper_tri_pca = corr_matrix_pca.values[np.triu_indices_from(corr_matrix_pca.values, k=1)]

# Create comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribution histogram - Original
axes[0, 0].hist(upper_tri_original, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0, 0].axvline(upper_tri_original.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {upper_tri_original.mean():.3f}')
axes[0, 0].axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.3)
axes[0, 0].set_title(f'Correlation Distribution - Original K-mer Features\n(Mean |r|: {np.abs(upper_tri_original).mean():.3f})', 
                     fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Correlation Coefficient', fontsize=10)
axes[0, 0].set_ylabel('Frequency', fontsize=10)
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Distribution histogram - PCA
axes[0, 1].hist(upper_tri_pca, bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[0, 1].axvline(upper_tri_pca.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {upper_tri_pca.mean():.3f}')
axes[0, 1].axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.3)
axes[0, 1].set_title(f'Correlation Distribution - PCA Components\n(Mean |r|: {np.abs(upper_tri_pca).mean():.3f})', 
                     fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Correlation Coefficient', fontsize=10)
axes[0, 1].set_ylabel('Frequency', fontsize=10)
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. KDE plot comparison
axes[1, 0].hist(upper_tri_original, bins=50, alpha=0.5, color='steelblue', label='Original', density=True)
axes[1, 0].hist(upper_tri_pca, bins=50, alpha=0.5, color='coral', label='PCA', density=True)
axes[1, 0].set_title('Overlay Comparison - Correlation Distributions', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Correlation Coefficient', fontsize=10)
axes[1, 0].set_ylabel('Density', fontsize=10)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Boxplot comparison
data_to_plot = [np.abs(upper_tri_original), np.abs(upper_tri_pca)]
box = axes[1, 1].boxplot(data_to_plot, labels=['Original', 'PCA'], patch_artist=True,
                         boxprops=dict(facecolor='lightblue', alpha=0.7),
                         medianprops=dict(color='red', linewidth=2))
axes[1, 1].set_title('Absolute Correlation - Boxplot Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('|Correlation Coefficient|', fontsize=10)
axes[1, 1].grid(alpha=0.3, axis='y')

# Color boxes
colors = ['steelblue', 'coral']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

plt.tight_layout()

# Save figure
output_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_distribution_comparison.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"💾 Figure saved: {output_path}")

plt.show()

## **10.9 Statistical Analysis of Correlations**

In [ ]:
print("="*80)
print("📊 STATISTICAL COMPARISON: ORIGINAL vs PCA FEATURES")
print("="*80)

# Calculate statistics
stats_original = {
    'Mean Correlation': upper_tri_original.mean(),
    'Mean Absolute Correlation': np.abs(upper_tri_original).mean(),
    'Median Correlation': np.median(upper_tri_original),
    'Std Correlation': upper_tri_original.std(),
    'Max Correlation': upper_tri_original.max(),
    'Min Correlation': upper_tri_original.min(),
    'High Corr (|r| > 0.7)': f"{(np.abs(upper_tri_original) > 0.7).sum() / len(upper_tri_original) * 100:.2f}%",
    'Moderate Corr (0.5 < |r| < 0.7)': f"{((np.abs(upper_tri_original) > 0.5) & (np.abs(upper_tri_original) <= 0.7)).sum() / len(upper_tri_original) * 100:.2f}%"
}

stats_pca = {
    'Mean Correlation': upper_tri_pca.mean(),
    'Mean Absolute Correlation': np.abs(upper_tri_pca).mean(),
    'Median Correlation': np.median(upper_tri_pca),
    'Std Correlation': upper_tri_pca.std(),
    'Max Correlation': upper_tri_pca.max(),
    'Min Correlation': upper_tri_pca.min(),
    'High Corr (|r| > 0.7)': f"{(np.abs(upper_tri_pca) > 0.7).sum() / len(upper_tri_pca) * 100:.2f}%",
    'Moderate Corr (0.5 < |r| < 0.7)': f"{((np.abs(upper_tri_pca) > 0.5) & (np.abs(upper_tri_pca) <= 0.7)).sum() / len(upper_tri_pca) * 100:.2f}%"
}

# Create comparison dataframe
df_stats = pd.DataFrame({
    'Metric': stats_original.keys(),
    'Original K-mer': stats_original.values(),
    'PCA Components': stats_pca.values()
})

print("\n📋 Correlation Statistics:")
print(df_stats.to_string(index=False))

print("\n" + "="*80)
print("📈 DIMENSIONALITY REDUCTION IMPACT")
print("="*80)
print(f"Original dimensions:        {df_original.shape[1]:,} features")
print(f"Reduced dimensions:         {df_pca.shape[1]:,} components")
print(f"Reduction ratio:            {(1 - df_pca.shape[1]/df_original.shape[1])*100:.1f}%")
print(f"Multicollinearity reduction: {(1 - np.abs(upper_tri_pca).mean()/np.abs(upper_tri_original).mean())*100:.1f}%")

print("\n" + "="*80)
print("✅ CONCLUSION")
print("="*80)
print("PCA successfully achieved:")
print("  ✓ Orthogonal components (near-zero correlations)")
print("  ✓ Reduced dimensionality while preserving information")
print("  ✓ Eliminated multicollinearity in feature space")
print("  ✓ Improved computational efficiency for downstream ML models")
print("="*80)

# Save statistics to CSV
stats_output_path = f"/home/tirta/Dokumen/FASTA-KmerReduce/rki_2025/reduction/{level_used}/{k_used}/{method_used}/correlation_statistics.csv"
df_stats.to_csv(stats_output_path, index=False)
print(f"\n💾 Statistics saved: {stats_output_path}")

## **10.10 Summary and Export Results**